In [1]:

import gym
import numpy as np
from utils import plot_learning_curve
import sys
from gym import spaces
import copy
import random
from datetime import datetime
import logging as log
import time
from pathlib import Path
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.callbacks import BaseCallback, EvalCallback,CheckpointCallback,CallbackList
import torch as th
import torch.nn as nn
import os
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor
cwd = os.getcwd()
print(cwd)
# Now you can use 'cwd' as the base path to construct any further paths
parent_dir = str(Path(cwd).parent)
sys.path.append(parent_dir)
from transferService import transferService
from optimizer_gd import *
from transferClass import *
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from stable_baselines3.common.monitor import Monitor

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-9yto6ct7 because the default path (/home/cc/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


/home/cc/transferService/final_code_transferService/ppo_4m_scratch


In [7]:
class NormalizeObservationAndRewardWrapper(gym.Wrapper):
    def __init__(self, env, obs_mean, obs_std, reward_scale=1.0):
        super().__init__(env)
        self.obs_mean = obs_mean
        self.obs_std = obs_std
        self.reward_scale = reward_scale
        self.old_score=0
        self.score_difference_positive_threadhold=2
        self.score_difference_negative_threadhold=-2
        self.observation_list=[]

    def reset(self, **kwargs):
        observation = self.env.reset(**kwargs)
        return observation

    def step(self, action):
        # observation, reward, done, _, info = self.env.step(action)
        observation, reward, done, info = self.env.step(action)
        self.observation_list.append(observation)
        normalized_obs = self.normalize_observation(observation)
        if reward != 1000000:
        #   normalized_reward = reward * self.reward_scale
            score_difference = reward - self.old_score
            self.old_score = reward
            if score_difference > self.score_difference_positive_threadhold:
                normalized_reward = 1
            elif score_difference < self.score_difference_negative_threadhold:
                normalized_reward = -1
            else:
                normalized_reward = 0
        else:
            normalized_reward = 5
        # return normalized_obs, normalized_reward, done, _ ,info
        return normalized_obs, normalized_reward, done, info

    def normalize_observation(self, observation):
      # try:
      #   return (observation - self.obs_mean) / self.obs_std
      # except:
      #   return np.zeros(40,)
        EPSILON = 1e-10  # Small constant to prevent division by zero
        normalized_observation = (observation - self.obs_mean) / (self.obs_std + EPSILON)
        return normalized_observation




def main():
    env_string = 'transferService'
    optimizer='ppo_sb3_10s_reward_by_difference'
    for handler in log.root.handlers[:]:
        log.root.removeHandler(handler)
    log_FORMAT = '%(created)f -- %(levelname)s: %(message)s'
    extraString="logFile"
      # Create the directory if it doesn't exist
    directory = f"./logFileDir/{optimizer}/"
    os.makedirs(directory, exist_ok=True)
    log_file = f"logFileDir/{optimizer}/{optimizer}_{extraString}_{datetime.now().strftime('%m_%d_%Y_%H_%M_%S')}.log"
    log.basicConfig(
        format=log_FORMAT,
        datefmt='%m/%d/%Y %I:%M:%S %p',
        level=log.INFO,
        handlers=[
            log.FileHandler(log_file),
            # log.StreamHandler()
        ]
    )
    REMOTE_IP = "129.114.109.46"
    REMOTE_PORT = "80"
    INTERVAL = 1
    INTERFACE = "eno1"
    SERVER_IP = '127.0.0.1'
    SERVER_PORT = 8080
    transfer_service = transferService(REMOTE_IP, REMOTE_PORT, INTERVAL, INTERFACE, SERVER_IP, SERVER_PORT, optimizer, log)
    env = transferClass(transfer_service,optimizer)

    data = np.load('obs_stats.npz')
    obs_mean = data['mean']
    obs_std = data['std']

    print("obs_mean after loading", obs_mean)
    print("obs_std after loading", obs_std)

    env = NormalizeObservationAndRewardWrapper(env, obs_mean, obs_std, reward_scale=1.0)

    policy_kwargs = dict(activation_fn=th.nn.ReLU,net_arch=[{'pi': [128, 128], 'vf': [128, 128]}])
    # model = PPO("MlpPolicy", env=env, policy_kwargs=policy_kwargs, verbose=1,tensorboard_log="./ppo_tensorboard/",ent_coef=0.01)


    # model = PPO.load("./ppo_checkpoints/ppo_model_4000_steps.zip")
    
    model = PPO.load("/home/cc/transferService/final_code_transferService/ppo_4m_scratch/ppo_best_model_reward_total/best_model")
    done = False
    episode_reward = 0
    obs = env.reset()


    while not done:
        action, _ = model.predict(obs, deterministic=True)
        # action = env.action_space.sample()
        print("action: ",action)
        # obs, reward, done, info = env.step([action])
        obs, reward, done, info = env.step(action)
        print("obs: ", obs,".... reward: ",reward)
        # episode_reward += reward[0]
        episode_reward += reward

    env.close()
    print(f"Episode reward: {episode_reward}")
    print("env.observation_list: ",env.observation_list)
if __name__ == '__main__':
    run_num = 0
    while run_num < 1:
        main()
        run_num += 1



obs_mean after loading [ 3.18628571e+00  3.68571429e+00  3.68571429e+00  0.00000000e+00
 -3.78571429e-01  2.82478571e+01  5.25285714e+01  9.51980645e-02
  3.74857143e+00  3.68571429e+00  3.68571429e+00  0.00000000e+00
 -8.75000000e+00  3.05271429e+01  5.25285714e+01  1.89282237e-01
  2.96914286e+00  3.68571429e+00  3.68571429e+00  0.00000000e+00
  6.14285714e-01  3.07728571e+01  5.25285714e+01  7.06785062e-02
  2.95771429e+00  3.68571429e+00  3.68571429e+00  0.00000000e+00
 -3.32857143e+00  3.05342857e+01  5.25285714e+01  8.00705097e-02
  3.10628571e+00  3.68571429e+00  3.68571429e+00  0.00000000e+00
 -8.92071429e+01  3.04871429e+01  5.27142857e+01  1.67036234e+00]
obs_std after loading [3.84853745e+00 2.56371860e+00 2.56371860e+00 0.00000000e+00
 9.79465441e+00 1.15687786e+01 3.33392610e+01 3.58803748e-01
 4.08249304e+00 2.56371860e+00 2.56371860e+00 0.00000000e+00
 7.39390797e+01 8.49246593e+00 3.33392610e+01 1.05087236e+00
 3.40531465e+00 2.56371860e+00 2.56371860e+00 0.00000000e+00

In [9]:
observation_list=[[ 0. ,  1. ,  1. ,  0. ,  0. , 32.7,  0. ,  0. ,  0. ,  1. ,  1. ,
        0. ,  0. , 32.8,  0. ,  0. ,  0. ,  1. ,  1. ,  0. ,  0. , 32.9,
        0. ,  0. ,  0. ,  1. ,  1. ,  0. ,  0. , 32.7,  0. ,  0. ,  0. ,
        1. ,  1. ,  0. ,  0. , 32.7,  0. ,  0. ],[3.20000000e-01, 8.00000000e+00, 8.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 5.83000000e+01, 0.00000000e+00, 0.00000000e+00,
       3.20000000e-01, 8.00000000e+00, 8.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 3.44000000e+01, 0.00000000e+00, 3.15613997e-03,
       0.00000000e+00, 8.00000000e+00, 8.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 3.49000000e+01, 0.00000000e+00, 4.47748475e-04,
       1.44000000e+01, 8.00000000e+00, 8.00000000e+00, 0.00000000e+00,
       4.00000000e+00, 3.28000000e+01, 0.00000000e+00, 0.00000000e+00,
       5.44000000e+00, 8.00000000e+00, 8.00000000e+00, 0.00000000e+00,
       2.00000000e+00, 3.30000000e+01, 0.00000000e+00, 1.67420906e-04], [5.44000000e+00, 8.00000000e+00, 8.00000000e+00, 0.00000000e+00,
       2.00000000e+00, 3.31000000e+01, 0.00000000e+00, 1.36784352e-04,
       3.20000000e+00, 8.00000000e+00, 8.00000000e+00, 0.00000000e+00,
       1.00000000e+00, 3.31000000e+01, 0.00000000e+00, 1.35866181e-05,
       1.34400000e+01, 8.00000000e+00, 8.00000000e+00, 0.00000000e+00,
       4.00000000e+00, 3.33000000e+01, 0.00000000e+00, 0.00000000e+00,
       5.12000000e+00, 8.00000000e+00, 8.00000000e+00, 0.00000000e+00,
       1.00000000e+00, 3.34000000e+01, 0.00000000e+00, 5.23305196e-05,
       1.02400000e+01, 8.00000000e+00, 8.00000000e+00, 0.00000000e+00,
       3.00000000e+00, 3.33000000e+01, 0.00000000e+00, 0.00000000e+00], [5.44000000e+00, 8.00000000e+00, 8.00000000e+00, 0.00000000e+00,
       1.00000000e+00, 3.29000000e+01, 0.00000000e+00, 9.26750377e-04,
       8.32000000e+00, 8.00000000e+00, 8.00000000e+00, 0.00000000e+00,
       2.00000000e+00, 3.58000000e+01, 0.00000000e+00, 0.00000000e+00,
       7.04000000e+00, 8.00000000e+00, 8.00000000e+00, 0.00000000e+00,
       2.00000000e+00, 3.35000000e+01, 0.00000000e+00, 0.00000000e+00,
       7.68000000e+00, 8.00000000e+00, 8.00000000e+00, 0.00000000e+00,
       2.00000000e+00, 3.30000000e+01, 0.00000000e+00, 0.00000000e+00,
       8.00000000e+00, 8.00000000e+00, 8.00000000e+00, 0.00000000e+00,
       2.00000000e+00, 3.41000000e+01, 0.00000000e+00, 0.00000000e+00], [1.60000000e+00, 8.00000000e+00, 8.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 3.75000000e+01, 0.00000000e+00, 7.53779549e-05,
       1.34400000e+01, 8.00000000e+00, 8.00000000e+00, 0.00000000e+00,
       4.00000000e+00, 3.29000000e+01, 0.00000000e+00, 0.00000000e+00,
       6.72000000e+00, 8.00000000e+00, 8.00000000e+00, 0.00000000e+00,
       2.00000000e+00, 3.27000000e+01, 0.00000000e+00, 0.00000000e+00,
       1.08800000e+01, 8.00000000e+00, 8.00000000e+00, 0.00000000e+00,
       3.00000000e+00, 3.27000000e+01, 0.00000000e+00, 0.00000000e+00,
       4.80000000e+00, 8.00000000e+00, 8.00000000e+00, 0.00000000e+00,
       1.00000000e+00, 3.27000000e+01, 0.00000000e+00, 0.00000000e+00],[ 0.32,  8.  ,  8.  ,  0.  ,  0.  , 32.8 ,  0.  ,  0.  ,  0.32,
        8.  ,  8.  ,  0.  ,  0.  , 32.7 ,  0.  ,  0.  ,  0.32,  8.  ,
        8.  ,  0.  ,  0.  , 32.8 ,  0.  ,  0.  ,  0.32,  8.  ,  8.  ,
        0.  ,  0.  , 32.8 ,  0.  ,  0.  ,  0.32,  8.  ,  8.  ,  0.  ,
        0.  , 32.8 ,  0.  ,  0.  ], [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0.]]

In [15]:
print(len(observation_list))

7


In [12]:
import numpy as np
import pandas as pd

# Define the function to reshape the list and create a DataFrame
def reshape_to_df(observation, column_names, index_names):
    # Reshape the observation to have 5 columns
    reshaped_data = np.reshape(observation, (5, -1)).T  # 5 columns, transpose to make them columns in df
    
    # Create the DataFrame with the specified index and column names
    df = pd.DataFrame(reshaped_data, columns=column_names, index=index_names)
    
    return df



# Desired column names
column_names = ['t1', 't2', 't3', 't4', 't5']

# Desired index names
index_names = ['throughput', 'parallelism', 'concurrency', 'lossRate', 'Score', 'rtt', 'energy', 's_lossRate']

# Call the function with the first observation
df = reshape_to_df(observation_list[0], column_names, index_names)

# Display the DataFrame
df


,t1,t2,t3,t4,t5
throughput,0.0,0.0,0.0,0.0,0.0
parallelism,1.0,1.0,1.0,1.0,1.0
concurrency,1.0,1.0,1.0,1.0,1.0
lossRate,0.0,0.0,0.0,0.0,0.0
Score,0.0,0.0,0.0,0.0,0.0
rtt,32.7,32.8,32.9,32.7,32.7
energy,0.0,0.0,0.0,0.0,0.0
s_lossRate,0.0,0.0,0.0,0.0,0.0


In [13]:
# Call the function with the first observation
df = reshape_to_df(observation_list[1], column_names, index_names)

# Display the DataFrame
df

,t1,t2,t3,t4,t5
throughput,0.32,0.320000,0.000000,14.4,5.440000
parallelism,8.00,8.000000,8.000000,8.0,8.000000
concurrency,8.00,8.000000,8.000000,8.0,8.000000
lossRate,0.00,0.000000,0.000000,0.0,0.000000
Score,0.00,0.000000,0.000000,4.0,2.000000
rtt,58.30,34.400000,34.900000,32.8,33.000000
energy,0.00,0.000000,0.000000,0.0,0.000000
s_lossRate,0.00,0.003156,0.000448,0.0,0.000167


In [14]:
# Call the function with the first observation
df = reshape_to_df(observation_list[2], column_names, index_names)

# Display the DataFrame
df

,t1,t2,t3,t4,t5
throughput,5.440000,3.200000,13.44,5.120000,10.24
parallelism,8.000000,8.000000,8.00,8.000000,8.00
concurrency,8.000000,8.000000,8.00,8.000000,8.00
lossRate,0.000000,0.000000,0.00,0.000000,0.00
Score,2.000000,1.000000,4.00,1.000000,3.00
rtt,33.100000,33.100000,33.30,33.400000,33.30
energy,0.000000,0.000000,0.00,0.000000,0.00
s_lossRate,0.000137,0.000014,0.00,0.000052,0.00


In [16]:
# Call the function with the first observation
df = reshape_to_df(observation_list[3], column_names, index_names)

# Display the DataFrame
df

,t1,t2,t3,t4,t5
throughput,5.440000,8.32,7.04,7.68,8.0
parallelism,8.000000,8.00,8.00,8.00,8.0
concurrency,8.000000,8.00,8.00,8.00,8.0
lossRate,0.000000,0.00,0.00,0.00,0.0
Score,1.000000,2.00,2.00,2.00,2.0
rtt,32.900000,35.80,33.50,33.00,34.1
energy,0.000000,0.00,0.00,0.00,0.0
s_lossRate,0.000927,0.00,0.00,0.00,0.0


In [17]:
# Call the function with the first observation
df = reshape_to_df(observation_list[4], column_names, index_names)

# Display the DataFrame
df

,t1,t2,t3,t4,t5
throughput,1.600000,13.44,6.72,10.88,4.8
parallelism,8.000000,8.00,8.00,8.00,8.0
concurrency,8.000000,8.00,8.00,8.00,8.0
lossRate,0.000000,0.00,0.00,0.00,0.0
Score,0.000000,4.00,2.00,3.00,1.0
rtt,37.500000,32.90,32.70,32.70,32.7
energy,0.000000,0.00,0.00,0.00,0.0
s_lossRate,0.000075,0.00,0.00,0.00,0.0


In [18]:
# Call the function with the first observation
df = reshape_to_df(observation_list[5], column_names, index_names)

# Display the DataFrame
df

,t1,t2,t3,t4,t5
throughput,0.32,0.32,0.32,0.32,0.32
parallelism,8.00,8.00,8.00,8.00,8.00
concurrency,8.00,8.00,8.00,8.00,8.00
lossRate,0.00,0.00,0.00,0.00,0.00
Score,0.00,0.00,0.00,0.00,0.00
rtt,32.80,32.70,32.80,32.80,32.80
energy,0.00,0.00,0.00,0.00,0.00
s_lossRate,0.00,0.00,0.00,0.00,0.00


In [19]:
# Call the function with the first observation
df = reshape_to_df(observation_list[6], column_names, index_names)

# Display the DataFrame
df

,t1,t2,t3,t4,t5
throughput,0.0,0.0,0.0,0.0,0.0
parallelism,0.0,0.0,0.0,0.0,0.0
concurrency,0.0,0.0,0.0,0.0,0.0
lossRate,0.0,0.0,0.0,0.0,0.0
Score,0.0,0.0,0.0,0.0,0.0
rtt,0.0,0.0,0.0,0.0,0.0
energy,0.0,0.0,0.0,0.0,0.0
s_lossRate,0.0,0.0,0.0,0.0,0.0
